## Configuration
a. stein 6.27.2023

Development of a configuration file and workflow

- clip data to region
- create drought network
    - network threshold
    - minimum area (area threshold)
- extract drought tracks
    - ratio threshold (s_f > ratio_thresh*s)
    

In [1]:
import yaml

In [2]:
def read_yaml(file_path):
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

In [3]:
names_yaml = """
- 'eric'
- 'justin'
- 'mary-kate'
"""

names = yaml.safe_load(names_yaml)

with open('names.yaml', 'w') as file:
    yaml.dump(names, file)

print(open('names.yaml').read())

- eric
- justin
- mary-kate



In [4]:
read_yaml('names.yaml')

['eric', 'justin', 'mary-kate']

In [5]:
ex_settings = read_yaml('example_yaml.yaml')
ex_settings

{'APP': {'ENVIRONMENT': 'test', 'DEBUG': True},
 'DATABASE': {'USERNAME': 'xiaoxu',
  'PASSWORD': 'xiaoxu',
  'HOST': '127.0.0.1',
  'PORT': 5432,
  'DB': 'xiaoxu_database'}}

In [6]:
ex_settings['DATABASE']

{'USERNAME': 'xiaoxu',
 'PASSWORD': 'xiaoxu',
 'HOST': '127.0.0.1',
 'PORT': 5432,
 'DB': 'xiaoxu_database'}

In [7]:
print(yaml.dump(ex_settings))

APP:
  DEBUG: true
  ENVIRONMENT: test
DATABASE:
  DB: xiaoxu_database
  HOST: 127.0.0.1
  PASSWORD: xiaoxu
  PORT: 5432
  USERNAME: xiaoxu



In [8]:
def write_config(
  config_path,
  data_dir, dnet_dir, track_dir, log_dir,
  metric_thresh, area_thresh, ratio_thresh      
):

    config = {
        "DIR":{
            "DATA":data_dir,
            "DNET":dnet_dir,
            "TRACK": track_dir,
            "LOG":log_dir,
        },
        "THRESH":{
            "METRIC":metric_thresh,
            "AREA":area_thresh,
            "RATIO":ratio_thresh,
        }
    }

    with open(config_path, 'w') as f:
        yaml.dump(config, f)
    

Let's now setup our config files

In [9]:
thresh_exp_dir = '/pool0/home/steinadi/data/drought/drought_impact/data/thresh_experiments'

In [10]:
spi_dir = '/pool0/home/steinadi/data/drought/drought_impact/data/drought_measures/spi/CONUS_105W/spi_usdmcat_CONUS_105W.nc'

In [11]:
import xarray as xr

In [12]:
spi = xr.open_dataset(spi_dir)

In [13]:
spi['spi_30d'].to_netcdf(f'{thresh_exp_dir}/spi30d/spi30d_usdmcat_CONUS_105W.nc')

In [14]:
xr.open_dataarray(f'{thresh_exp_dir}/spi30d/spi30d_usdmcat_CONUS_105W.nc')

<xarray.DataArray 'spi_30d' (time: 3066, y: 702, x: 525)>
[1129974300 values with dtype=float32]
Coordinates:
  * x        (x) float64 -2.893e+06 -2.889e+06 ... -6.674e+05 -6.632e+05
  * y        (y) float64 3.251e+06 3.246e+06 3.242e+06 ... 2.717e+05 2.674e+05
  * time     (time) datetime64[ns] 1980-01-05 1980-01-10 ... 2021-12-31
Attributes:
    units:              
    description:        Standardized Precipitation Index
    long_name:          spi30d
    standard_name:      spi30d
    dimensions:         lon lat time
    coordinate_system:  WGS84,EPSG:4326
    Clipping:           This selection has been clipped to everything west of...
    coordinates:        lon lat
    grid_mapping:       crs

In [ ]:
spi['spi_180d'].to_netcdf(f'{thresh_exp_dir}/spi180d/spi180d_usdmcat_CONUS_105W.nc')

In [16]:
import numpy as np

In [17]:
metric_thresh = 1
area_thresh = np.arange(0, 3000+200, 200)
ratio_thresh = np.arange(0, 1+0.1, 0.1)

In [13]:
import os

In [16]:
os.path.exists?

Signature: os.path.exists(path)
Docstring: Test whether a path exists.  Returns False for broken symbolic links
File:      /pool0/data/steinadi/.conda/envs/sedi/lib/python3.10/genericpath.py
Type:      function


In [19]:
for a_thresh in area_thresh:
    for r_thresh in ratio_thresh:

        key = f'{a_thresh}a_0{int(r_thresh*10)}r_{metric_thresh}m'

        if not os.path.exists(f'{thresh_exp_dir}/spi30d/config/config_{key}.yml'):
            write_config(
                config_path=f'{thresh_exp_dir}/spi30d/config/config_{key}.yml',
                data_dir=f'{thresh_exp_dir}/spi30d/spi30d_usdmcat_CONUS_105W.nc',
                dnet_dir=f'{thresh_exp_dir}/spi30d/dnet/net_{key}.pickle',
                track_dir=f'{thresh_exp_dir}/spi30d/track/track_{key}.pickle',
                log_dir=f'{thresh_exp_dir}/spi30d/log/log_{key}',
                metric_thresh=f'{metric_thresh}',
                area_thresh=f'{a_thresh}',
                ratio_thresh=f'{r_thresh}'
            )

In [34]:
read_yaml('/pool0/home/steinadi/data/drought/drought_impact/data/thresh_experiments/spi30d/config/config_0a_05r_1m.yml')

{'DIR': {'DATA': '/pool0/home/steinadi/data/drought/drought_impact/data/thresh_experiments/spi30d/spi30d_usdmcat_CONUS_105W.nc',
  'DNET': '/pool0/home/steinadi/data/drought/drought_impact/data/thresh_experiments/spi30d/dnet/net_0a_05r_1m.pickle',
  'LOG': '/pool0/home/steinadi/data/drought/drought_impact/data/thresh_experiments/spi30d/log/log_0a_05r_1m',
  'TRACK': '/pool0/home/steinadi/data/drought/drought_impact/data/thresh_experiments/spi30d/track/track_0a_05r_1m.pickle'},
 'THRESH': {'AREA': '0', 'METRIC': '1', 'RATIO': '0.5'}}